**ANTES DE PROSSEGUIR FAÇA UMA CÓPIA DESTE NOTEBOOK**

# SI10 | 2025 | T10 | Prova Prática

**Nome**: Daniel Mendez

----

## Contexto

Você foi contratado como analista de dados por uma empresa de e-commerce que está enfrentando desafios na conversão de visitantes em clientes.

Ao investigar os registros de navegação, a equipe de produto identificou gargalos no fluxo de páginas e propôs uma alteração estratégica no site no dia `2023-01-30`, com o objetivo de melhorar a taxa de conversão.

Seu trabalho consiste em analisar os dados de navegação antes e depois da intervenção, identificar padrões de comportamento dos usuários e responder se a mudança surtiu efeito.

## Importação do dataset e das bibliotecas

In [1]:
# Execute essas linhas de código

# Importação do dataset

# https://drive.google.com/file/d/1gAe0H_gv_rk_9wPnBgB1rzTHi_nnkpDc/view?usp=sharing
!gdown 1gAe0H_gv_rk_9wPnBgB1rzTHi_nnkpDc

Downloading...
From: https://drive.google.com/uc?id=1gAe0H_gv_rk_9wPnBgB1rzTHi_nnkpDc
To: /content/ITL-SI10-2025-Prova_Dataset.csv
100% 178k/178k [00:00<00:00, 77.4MB/s]


In [2]:
# Execute essas linhas de código

# Importação das bibliotecas
import pandas as pd

# Transforma o dataset em DataFrame Pandas
df = pd.read_csv('ITL-SI10-2025-Prova_Dataset.csv')
print(df.head())

   user_id session_id        data  passo   pagina
0      956     00_000  2023-01-01      1     Home
1      956     00_000  2023-01-01      2  Produto
2      956     00_000  2023-01-01      3    Saída
3      418     00_001  2023-01-01      1     Home
4      418     00_001  2023-01-01      2    Saída


## Questões

In [3]:
# Execute essas linhas de código

dia_mudanca = 30
df["data"] = pd.to_datetime(df["data"])
data_mudanca = pd.Timestamp("2023-01-01") + pd.Timedelta(days=dia_mudanca)

### Questão 1

**Matriz de Transição de Markov**. A empresa deseja entender como os usuários se comportavam antes da mudança no site. Para isso, utilize o conjunto de dados `df_antes`, que contém os registros de navegação anteriores ao dia `2023-01-30`.

Construa uma matriz de transição de Markov onde cada célula representa a probabilidade de um usuário transitar de uma página para outra dentro da mesma sessão.

In [4]:
# Execute essas linhas de código

df_antes = df[df["data"] < data_mudanca].copy()
df_antes.head()

,user_id,session_id,data,passo,pagina
0,956,00_000,2023-01-01,1,Home
1,956,00_000,2023-01-01,2,Produto
2,956,00_000,2023-01-01,3,Saída
3,418,00_001,2023-01-01,1,Home
4,418,00_001,2023-01-01,2,Saída


In [5]:
# Escreva sua resposta nesta seção (Você pode criar quantas células de texto e de código forem necessárias)

# Ordenação e criação de coluna de próxima página
df_antes = df_antes.sort_values(['session_id', 'passo'])
df_antes['next_page'] = df_antes.groupby('session_id')['pagina'].shift(-1)

# Cálculo das transições
transitions = df_antes.dropna(subset=['next_page'])
transition_counts = transitions.groupby(['pagina', 'next_page']).size().unstack(fill_value=0)

# Normalização para obter probabilidades
transition_matrix = transition_counts.div(transition_counts.sum(axis=1), axis=0)

# Exibição da matriz de transição
transition_matrix

next_page,Carrinho,Checkout,Obrigado,Produto,Saída
pagina,,,,,
Carrinho,0.000000,0.729097,0.0,0.000000,0.270903
Checkout,0.000000,0.000000,1.0,0.000000,0.000000
Home,0.000000,0.000000,0.0,0.618889,0.381111
Produto,0.536804,0.000000,0.0,0.000000,0.463196


**Interpretação das transições:**

1. **Home → Produto (61,89 %) ou Saída (38,11 %)**
   — Quando entram na “Home”, a maioria dos usuários (≈ 62 %) vai para a página de “Produto”, mas ainda há uma parcela significativa cobrindo abandono imediato (≈ 38 %).

2. **Produto → Carrinho (53,68 %) ou Saída (46,32 %)**
   — Em metade das vezes em que veem um produto, o usuário adiciona ao carrinho. Porém, quase metade sai do site direto da página de produto, sem continuar o fluxo.

3. **Carrinho → Checkout (72,91 %) ou Saída (27,09 %)**
   — Dos que chegam ao carrinho, cerca de 73 % prosseguem para a finalização (Checkout). Os outros 27 % ainda abandonam nessa etapa.

4. **Checkout → Obrigado (100 %)**
   — Todos os que iniciam o checkout chegam à página de “Obrigado” (confirmando a compra) — ou seja, não há fugas dentro do processo de pagamento propriamente dito.

* **Ponto de maior vazamento**:

  1. **Produto → Saída** (46 %)
  2. **Home → Saída** (38 %)
  3. **Carrinho → Saída** (27 %)

### Questão 2

**Simulação de Monte Carlo**: Simule a navegação de 1.000 usuários iniciando pela página Home, utilizando a matriz de transição de Markov para determinar os próximos passos..

📌 Regras da simulação:

- Cada simulação começa no estado `Home`.

- A navegação termina quando o usuário chega a um estado final: `Obrigado` (conversão) ou `Saída` (abandono).

- Use sorteios aleatórios baseados nas probabilidades da matriz para definir as transições.


Entregável:

- Número de usuários que chegaram até Obrigado.

- Número de usuários que chegaram até Saída.

- Taxa de conversão e taxa de abandono estimadas com base na simulação.

In [15]:
import numpy as np

In [16]:
# Converter em dicionário para simulação
transitions_dict = {}
for origin in transition_matrix.index:
    probs = transition_matrix.loc[origin]
    next_states = list(probs[probs > 0].index)
    probs_list = list(probs[probs > 0].values)
    transitions_dict[origin] = (next_states, probs_list)

# Simulação de Monte Carlo
num_simulations = 1000
outcomes = []

for _ in range(num_simulations):
    state = 'Home'
    while state not in ['Obrigado', 'Saída']:
        next_states, probs = transitions_dict[state]
        state = np.random.choice(next_states, p=probs)
    outcomes.append(state)

# Agregar resultados
counts = pd.Series(outcomes).value_counts().reindex(['Obrigado','Saída'], fill_value=0)
rates  = (counts / num_simulations).round(4)
df_simulation = pd.DataFrame({
    'Quantidade de Usuários': counts,
    'Taxa (proporção)': rates
})

print(df_simulation)

          Quantidade de Usuários  Taxa (proporção)
Obrigado                     219             0.219
Saída                        781             0.781


Na simulação com 1.000 usuários:

219 chegaram à página “Obrigado” → taxa de conversão de 21,9 %.

781 terminaram em “Saída” → taxa de abandono de 78,1 %.

### Questão 3

**Teste A/B**: A mudança proposta no site foi implantada no dia `2023-01-30`. Para avaliar seu impacto, você recebeu dois subconjuntos de dados:

Você recebeu dois conjuntos de dados:

- `df_antes`: registros de navegação dos usuários antes da mudança

- `df_depois`: registros após a mudança

Verifique se a mudança foi eficaz em melhorar o comportamento dos usuários.

In [ ]:
df_depois = df[df["data"] >= data_mudanca].copy()
df_depois.head()

,user_id,session_id,data,passo,pagina
2874,551,30_000,2023-01-31,1,Home
2875,551,30_000,2023-01-31,2,Produto
2876,551,30_000,2023-01-31,3,Carrinho
2877,551,30_000,2023-01-31,4,Saída
2878,355,30_001,2023-01-31,1,Home


In [18]:
from statsmodels.stats.proportion import proportions_ztest

In [19]:
# 1) Preparar os dois subconjuntos
df_antes = df[df["data"] < data_mudanca].copy()
df_depois = df[df["data"] >= data_mudanca].copy()

In [20]:
# 2) Extrair o outcome final de cada sessão
def session_outcome(subdf):
    last_page = subdf.sort_values("passo").iloc[-1]["pagina"]
    return 1 if last_page == "Obrigado" else 0  # 1=converteu, 0=abandonou

sess_antes = df_antes.groupby("session_id").apply(session_outcome)
sess_depois = df_depois.groupby("session_id").apply(session_outcome)

<ipython-input-20-1739346979>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sess_antes = df_antes.groupby("session_id").apply(session_outcome)
<ipython-input-20-1739346979>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sess_depois = df_depois.groupby("session_id").apply(session_outcome)


In [21]:
# 3) Calcular métricas
n1 = len(sess_antes)
conv1 = sess_antes.sum()
rate1 = conv1 / n1

n2 = len(sess_depois)
conv2 = sess_depois.sum()
rate2 = conv2 / n2

print(f"Antes:   {conv1}/{n1} → taxa de conversão = {rate1:.2%}")
print(f"Depois:  {conv2}/{n2} → taxa de conversão = {rate2:.2%}")

Antes:   218/900 → taxa de conversão = 24.22%
Depois:  234/900 → taxa de conversão = 26.00%


## Entrega

In [22]:
# 4) Teste de hipótese (z-test para duas proporções)
count = np.array([conv1, conv2])
nobs  = np.array([n1, n2])
stat, pval = proportions_ztest(count, nobs)

print(f"\nZ-statistic = {stat:.3f}, p-valor = {pval:.4f}")
if pval < 0.05:
    print("→ Diferença estatisticamente significativa (p < 0.05).")
else:
    print("→ Sem evidência estatística forte de mudança (p ≥ 0.05).")


Z-statistic = -0.870, p-valor = 0.3845
→ Sem evidência estatística forte de mudança (p ≥ 0.05).


A comparação mostra:

Antes: 218 de 900 sessões convertidas (24,22 %).

Depois: 234 de 900 sessões convertidas (26,00 %).

Teste Z: p-valor = 0,3845 (> 0,05), logo não há evidência estatística de que o aumento de 1,78 p.p. seja real e não apenas ruído.

Em resumo, embora a taxa de conversão tenha subido de 24,2 % para 26,0 %, a diferença não é estatisticamente significativa no nível de 5 %.

Siga esses passos para entregar sua prova.

1. Crie um repositório no GitHub para essa entrega.

2. Submeta seu desenvolvimento neste repositório.

3. Garanta que o repositório é publicamente acessível (eliminatório).

4. No Google forms da prova, submeta a URL do repositório.